In [1]:
import pandas as pd
import re

In [2]:
Xtr = pd.read_csv('Xtr.csv')
Xte = pd.read_csv('Xte.csv')

Xtr = Xtr[['ID','Address']]
Xte = Xte[['ID','Address']]

Xtr['Address'] = Xtr['Address'].astype(str)
Xte['Address'] = Xte['Address'].astype(str)

C:\Users\Tyler\AppData\Local\Temp\ipykernel_8944\1219892066.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Xte = pd.read_csv('Xte.csv')


In [10]:
# https://www.kaggle.com/code/msk1097/energy-star-score-regression
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
    
missing_values_table(Xtr)
missing_values_table(Xte)

Your selected dataframe has 2 columns.
There are 0 columns that have missing values.
Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [3]:
Xtr['Address'] = Xtr['Address'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)
Xte['Address'] = Xte['Address'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)
Xtr['Address'] = Xtr['Address'].str.lstrip()
Xte['Address'] = Xte['Address'].str.lstrip()
Xtr['Address'] = Xtr['Address'].str.replace(r"\s+[A-Z]$", "", regex=True)
Xte['Address'] = Xte['Address'].str.replace(r"\s+[A-Z]$", "", regex=True)
Xtr['Address'] = Xtr['Address'].str.replace(r'\s+', ' ', regex=True)
Xte['Address'] = Xte['Address'].str.replace(r'\s+', ' ', regex=True)

In [4]:
abbreviation_mapping = {
    'VLG': 'VILLAGE',
    'VILL': 'VILLAGE',
    'ST': 'STREET',
    'RD': 'ROAD',
    'LN': 'LANE',
    'LA' : 'LANE',
    'AVE': 'AVENUE',
    'AV': 'AVENUE',
    'RDG': 'RIDGE',
    'BLVD': 'BOULEVARD',
    'RTE': 'ROUTE',
    'UT': 'UNIT',
    'CIR': 'CIRCLE',
    'ISLE': 'IS',
    'CT' : 'COURT',
    'DR' : 'DRIVE',
    'PL' : 'PLACE',
    'HL' : 'HILL',
    'PT' : 'POINT',
    'TR' : 'TERRACE',
    'TER' : 'TERRACE',
    'TERR' : 'TERRACE',
    'HWY' : 'HIGHWAY',
    'HGHY' : 'HIGHWAY',
    'HGWY' : 'HIGHWAY',
    'TRL' : 'TRAIL',
    'SQ' : 'SQUARE',
    'LT' : 'LOT'
}
pattern = r'\b(' + '|'.join(abbreviation_mapping.keys()) + r')\b'
def replace_abbreviations(match):
    return abbreviation_mapping[match.group(0)]
Xtr['Address'] = Xtr['Address'].apply(lambda x: re.sub(pattern, replace_abbreviations, x))
Xte['Address'] = Xte['Address'].apply(lambda x: re.sub(pattern, replace_abbreviations, x))

In [5]:
direction_mapping = {
    'W ': 'WEST ',
    'E ': 'EAST ',
    'N ': 'NORTH ',
    'S ': 'SOUTH '
}
pattern = r'^(' + '|'.join(re.escape(key) for key in direction_mapping.keys()) + r')'

def replace_directions(match):
    return direction_mapping[match.group(0)]
Xtr['Address'] = Xtr['Address'].str.replace(pattern, replace_directions, regex=True)
Xte['Address'] = Xte['Address'].str.replace(pattern, replace_directions, regex=True)

In [6]:
pattern = r' (LOT|UNIT|EXT|EXTENSION)$'
Xtr['Address'] = Xtr['Address'].str.replace(pattern, r'\1', regex=True)
Xte['Address'] = Xte['Address'].str.replace(pattern, r'\1', regex=True)
pattern = r'^[A-C]\s'
Xtr['Address'] = Xtr['Address'].str.replace(pattern, '', regex=True)
Xte['Address'] = Xte['Address'].str.replace(pattern, '', regex=True)

In [7]:
pattern = r'(STREET|AVENUE|DRIVE|ROAD)(EXT|UNIT)$'

def remove_ext(match):
    return match.group(1)

Xtr['Address'] = Xtr['Address'].str.replace(pattern, remove_ext, regex=True)
Xte['Address'] = Xte['Address'].str.replace(pattern, remove_ext, regex=True)

In [8]:
Xtr.to_csv('Xtr_Address.csv', index=False)
Xte.to_csv('Xte_Address.csv', index=False)